In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace


flags = Namespace(
    train_file='data.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['I', 'am'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
)


def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.split()

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text


def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]


class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x.long())
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))


def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer


def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()
    words = ['I', 'am']

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])

    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words).encode('utf-8'))


def main():
    device = torch.device('cuda')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0

    for e in range(200):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            net.train()

            optimizer.zero_grad()

            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y.long())

            loss_value = loss.item()

            loss.backward()

            state_h = state_h.detach()
            state_c = state_c.detach()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                torch.save(net.state_dict(),
                           'model.pth'.format(iteration))


if __name__ == '__main__':
    main()

Vocabulary size 11233
Epoch: 0/200 Iteration: 100 Loss: 5.798356533050537
Epoch: 0/200 Iteration: 200 Loss: 5.379260063171387
Epoch: 0/200 Iteration: 300 Loss: 5.341028690338135
Epoch: 1/200 Iteration: 400 Loss: 5.085740089416504
Epoch: 1/200 Iteration: 500 Loss: 4.642521381378174
Epoch: 1/200 Iteration: 600 Loss: 4.922443389892578
Epoch: 1/200 Iteration: 700 Loss: 4.72169303894043
Epoch: 2/200 Iteration: 800 Loss: 4.502391815185547
Epoch: 2/200 Iteration: 900 Loss: 4.283120155334473
Epoch: 2/200 Iteration: 1000 Loss: 4.320672988891602
b'I am , I suppose you can be done to you ? He had only put it . " said Oliver Twist ; " I am , and I see him ! What do , my heart , for it , and that it might know it to me ? I don\'t see it , and , " replied Oliver Twist : not to be the two boys , " and Duff , " replied Mrs , with his head with an instant with his head , he had to him . He has been to a little room : the old gentleman'
Epoch: 2/200 Iteration: 1100 Loss: 4.353078842163086
Epoch: 3/200 I

Epoch: 23/200 Iteration: 8900 Loss: 3.3081889152526855
Epoch: 23/200 Iteration: 9000 Loss: 2.7887747287750244
b'I am going to see you , my eye , sir ! Come away ; I know , " stammered Sikes : \'so pray , Fagin ? An very difficult and death of the house t h you . What\'s her tears , and I know you talk for the money . However the law was not fair , " observed Master justice ! What are all these ( Sikes was a good of dust . If Mrs he were properly in his abode ; for it were in no loss in visiting wide whisper . \'the prerogative first one night ensued ,'
Epoch: 23/200 Iteration: 9100 Loss: 2.9395816326141357
Epoch: 23/200 Iteration: 9200 Loss: 3.117337226867676
Epoch: 24/200 Iteration: 9300 Loss: 2.8548645973205566
Epoch: 24/200 Iteration: 9400 Loss: 3.036689281463623
Epoch: 24/200 Iteration: 9500 Loss: 3.0205838680267334
Epoch: 24/200 Iteration: 9600 Loss: 3.087684154510498
Epoch: 25/200 Iteration: 9700 Loss: 2.8477370738983154
Epoch: 25/200 Iteration: 9800 Loss: 2.984802722930908
Epoch

Epoch: 43/200 Iteration: 16800 Loss: 2.728569269180298
Epoch: 44/200 Iteration: 16900 Loss: 2.9633615016937256
Epoch: 44/200 Iteration: 17000 Loss: 2.950498580932617
b'I am very lucky from your mouth and add . Lead , he was to inherit her . Mr was attired than an hour after water for the beadle . " Unless ? Mr , hungry ; for the Dodger\'s manner was , with an intimation up to the narrative a file man to a milk wrench on , nevertheless ; and licking the dog . At first a family on the chain , carefully wounded , for a few feet , and his face with extraordinary readiness quickly , that they would recoil from the grave or chimneys and impetuous gentleman with'
Epoch: 44/200 Iteration: 17100 Loss: 2.789227247238159
Epoch: 44/200 Iteration: 17200 Loss: 2.7255747318267822
Epoch: 45/200 Iteration: 17300 Loss: 2.6393139362335205
Epoch: 45/200 Iteration: 17400 Loss: 2.709343910217285
Epoch: 45/200 Iteration: 17500 Loss: 2.641984462738037
Epoch: 45/200 Iteration: 17600 Loss: 2.890205144882202
Epo

Epoch: 64/200 Iteration: 24600 Loss: 2.5666327476501465
Epoch: 64/200 Iteration: 24700 Loss: 2.8607959747314453
Epoch: 64/200 Iteration: 24800 Loss: 2.6847786903381348
Epoch: 64/200 Iteration: 24900 Loss: 2.635082244873047
Epoch: 65/200 Iteration: 25000 Loss: 2.730562686920166
b'I am swelling roared to know , " said Oliver to communicate with great attention with fatigue , who , after participating with a degree grew out a born struggling in the broth : " ; for he is . He would rather have been your mind ? " said Mr ; filling a lamp down to the prevailing cheerful ; " it is , with you . They had not strength for the boy ! Quick ain\'t such more as you don\'t know that , " replied Mr . Sikes , ruminating his arms . \'there ! " answered Charley : looking'
Epoch: 65/200 Iteration: 25100 Loss: 2.7236852645874023
Epoch: 65/200 Iteration: 25200 Loss: 2.769556760787964
Epoch: 65/200 Iteration: 25300 Loss: 2.65002179145813
Epoch: 66/200 Iteration: 25400 Loss: 2.6666951179504395
Epoch: 66/200 I

Epoch: 84/200 Iteration: 32500 Loss: 2.6135330200195312
Epoch: 84/200 Iteration: 32600 Loss: 2.960808753967285
Epoch: 85/200 Iteration: 32700 Loss: 2.4852678775787354
Epoch: 85/200 Iteration: 32800 Loss: 2.706228494644165
Epoch: 85/200 Iteration: 32900 Loss: 3.035757064819336
Epoch: 85/200 Iteration: 33000 Loss: 2.4460227489471436
b'I am very orphan , eh , I know the first time he is very bad . If they have heard enough , you must be sworn , you won\'t come at . I could never see you mind ! I must go back to see ? What were , dear lady to rights ! I have saved her abode than against the City , or I\'ll take a we\'re friend . I am going out ; " how differently occurred to night that I am very sorry if to you , " rejoined Monks , stooping down . He gazed violently ,'
Epoch: 86/200 Iteration: 33100 Loss: 2.4070160388946533
Epoch: 86/200 Iteration: 33200 Loss: 2.55824875831604
Epoch: 86/200 Iteration: 33300 Loss: 2.6258625984191895
Epoch: 86/200 Iteration: 33400 Loss: 2.7800827026367188
Ep

Epoch: 106/200 Iteration: 40800 Loss: 2.9304637908935547
Epoch: 106/200 Iteration: 40900 Loss: 2.603687047958374
Epoch: 106/200 Iteration: 41000 Loss: 2.826559066772461
b'I am afraid towards Fagin ? " asked Monks without the driver to prepare , " of this , the old hag the robbery . Quit anything if you behave look at his young , and mistaking a glimpse heart ; the boy comes to this determination ; that\'s plain . He answered Mr : weak in the darkness ; and in view of buildings on the bed , rendered his whole old gentleman . " I have been so glad . I will never know . " The gentleman in this river doubts as though to possess ; for a short bottle'
Epoch: 107/200 Iteration: 41100 Loss: 2.8431451320648193
Epoch: 107/200 Iteration: 41200 Loss: 2.7274343967437744
Epoch: 107/200 Iteration: 41300 Loss: 2.6863722801208496
Epoch: 107/200 Iteration: 41400 Loss: 2.839228868484497
Epoch: 108/200 Iteration: 41500 Loss: 2.7178738117218018
Epoch: 108/200 Iteration: 41600 Loss: 2.7097902297973633
Epoch

Epoch: 126/200 Iteration: 48500 Loss: 2.6399471759796143
Epoch: 126/200 Iteration: 48600 Loss: 2.680476188659668
Epoch: 126/200 Iteration: 48700 Loss: 2.7937803268432617
Epoch: 127/200 Iteration: 48800 Loss: 2.6022071838378906
Epoch: 127/200 Iteration: 48900 Loss: 2.8360490798950195
Epoch: 127/200 Iteration: 49000 Loss: 2.7590036392211914
b'I am equally book backward , " replied Mrs , breathless Oliver ; as the sound stood hastily , in his alarm . To , he turned a quickly . Oliver paced next , damp and feeble candle , by a furious chair across his hair ; the feeble candle was right . The month\'s them as a shade above Mr . John , administered a chairman , with an air hastily discharge of five doors ; but uttering a profound low state and chair at each process of his eye as he could , on the old gentleman\'s commencement , as the'
Epoch: 127/200 Iteration: 49100 Loss: 2.648635149002075
Epoch: 128/200 Iteration: 49200 Loss: 2.5965802669525146
Epoch: 128/200 Iteration: 49300 Loss: 2.83790

Epoch: 146/200 Iteration: 56100 Loss: 2.633755683898926
Epoch: 146/200 Iteration: 56200 Loss: 2.7616055011749268
Epoch: 146/200 Iteration: 56300 Loss: 2.791590929031372
Epoch: 146/200 Iteration: 56400 Loss: 2.613173723220825
Epoch: 147/200 Iteration: 56500 Loss: 2.807941198348999
Epoch: 147/200 Iteration: 56600 Loss: 2.7872414588928223
Epoch: 147/200 Iteration: 56700 Loss: 2.6196095943450928
Epoch: 147/200 Iteration: 56800 Loss: 2.5822300910949707
Epoch: 148/200 Iteration: 56900 Loss: 2.9332377910614014
Epoch: 148/200 Iteration: 57000 Loss: 2.330857753753662
b'I am going out of woollen or country : not very much afraid . You " Your father being spared the humour , and , on my own life ! What have been a pity the belief . " Mr , aunt , as he advanced with the porter and red nose towards Nancy . Sikes had been completely over in their condition in earnest ; and they had strayed . The old man had been torn working with a curiousity of a low manner , as they came from his usual life which 

Epoch: 166/200 Iteration: 64100 Loss: 2.6526217460632324
Epoch: 167/200 Iteration: 64200 Loss: 2.788362503051758
Epoch: 167/200 Iteration: 64300 Loss: 2.8032262325286865
Epoch: 167/200 Iteration: 64400 Loss: 2.744192361831665
Epoch: 167/200 Iteration: 64500 Loss: 2.7998342514038086
Epoch: 168/200 Iteration: 64600 Loss: 2.6559810638427734
Epoch: 168/200 Iteration: 64700 Loss: 2.700021743774414
Epoch: 168/200 Iteration: 64800 Loss: 2.568584680557251
Epoch: 169/200 Iteration: 64900 Loss: 2.8599891662597656
Epoch: 169/200 Iteration: 65000 Loss: 2.830838918685913
b'I am weak , " whimpered Oliver , \'to have a slow pavement , " did it ! Listen ! How should I read it . I would have done a vast deal , for many people , and you can be the truest ? " said the girl placing a field . At the mansion , from a plot to take care , and shielding a very existence to retrieve this juncture ; then , in the earlier months and thirty , of an unusual speed ; rendered it , climbed it ; " he cried only one man

Epoch: 187/200 Iteration: 72100 Loss: 2.8968329429626465
Epoch: 188/200 Iteration: 72200 Loss: 2.812753915786743
Epoch: 188/200 Iteration: 72300 Loss: 3.0596394538879395
Epoch: 188/200 Iteration: 72400 Loss: 2.680244207382202
Epoch: 188/200 Iteration: 72500 Loss: 2.6497910022735596
Epoch: 189/200 Iteration: 72600 Loss: 2.4875662326812744
Epoch: 189/200 Iteration: 72700 Loss: 2.830587148666382
Epoch: 189/200 Iteration: 72800 Loss: 2.5951709747314453
Epoch: 189/200 Iteration: 72900 Loss: 2.598219156265259
Epoch: 190/200 Iteration: 73000 Loss: 2.727513313293457
b'I am \'thank get through your own mind , to you know how devotedly I loved him . " " Ah so ! She ain\'t less equal a lifer in his own own heart , which I wants to deny that he had been a very spent , for I feel _I_ and see that this place was not , in that frightened all your goodness on the same ; and , on your friend ; but , I am to prove to be gone seven at that a spring . Yer change that , oh , the girls\'s repeat , and share